In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 64
batch_size = 128
max_iters = 1000
learning_rate = 3e-3
eval_iters = 200
n_embd = 384 # Stores the vectors of the token
n_head = 4
n_layer = 4 # Number of decoder blocks
dropout = 0.2

cuda


In [29]:
chars = ""
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [30]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

data = torch.tensor(encode(text), dtype=torch.long)

In [31]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [32]:
@torch.no_grad() # Ensure PyTorch does not use gradients at all, this reduces computations, better performance
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Positional Encoding

Positional Encoding (PE) injects information about the token positions into the permutation-invariant Transformer model.

The Positional Encoding Function

The sinusoidal positional encoding vector $\text{PE}$ for a token at position $pos$ and dimension $i$ is calculated as follows:

$$\text{PE}(pos, 2i) = \sin \left( \frac{pos}{10000^{2i/d_{model}}} \right)$$

$$\text{PE}(pos, 2i+1) = \cos \left( \frac{pos}{10000^{2i/d_{model}}} \right)$$

Where:

$pos$ is the token's position in the sequence ($0, 1, 2, \dots$).

$i$ is the index for the dimension pairs ($0, 1, \dots, d_{model}/2 - 1$).

$d_{model}$ is the embedding dimension.

This encoding is simply added to the input embedding $x$ before it is processed by the Transformer layers:

$$\text{Input to Transformer} = x + \text{PE}$$

In [ ]:
class Head(nn.Module):
    """Single head of multi-head"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # input size (batch, time-step, channels)
        # output size (batch, time-step, head-size)
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute attention scores
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # We are calculating Scaled Score = (Query x Key(Transpose)) x 1/√d, where 'd' is the dimension of the Q and K vectors.
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # We mask the future results so that the model not look into the future, using upper triangle inf matrix
        wei = wei.softmax(dim=-1) # Applying softmax to get the probability distribution
        wei = self.dropout(wei) # Avoid overfitting
        # perform weighted aggregation of the values
        v = self.value(x) # The values
        out = wei @ v # Matmul on Attention Weight and Values
        return out # Finally we get the output
    

class MultiHeadAttention(nn.Module):
    """Multiple heads of Self-Attention in parallel"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # Each head computing in parallel
        self.proj = nn.Linear(head_size * num_heads, n_embd) # After each head completes process, combine those into single vector
        self.dropout = nn.Dropout(dropout) # Avoid overfitting
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """Simple linear layer followed by non-linearity"""
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(), # ReLu = max(0, x)
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout) # drop few neurons for better learning and avoid overfitting
        )
    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """Transformer Block: Communication follower by computation"""
    
    def __init__(self, n_embd, n_head):
        # n_embd: Embedding Dimensions, n_head: number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) # self-attention
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) # Layer Normalization
        self.ln2 = nn.LayerNorm(n_embd) # Layer Normalization
    
    # Self-Attention -> Normalization -> FeedForward -> Normalization
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

# GPT Architechture, Just a little scaled down version of Transformer Architechture
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)
    
    # Weight Initialization using S.D, Goal: Avoid Gradient Explosion or Vanish
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    # Forward Pass
    def forward(self, index, targets=None):
        B, T = index.shape

        # index and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)
            logits = logits[:, -1, :]  # (B, C)
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"steps: {iter}, train loss: {losses['train']:.4f}, validation loss: {losses['val']:.4f}")

    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

steps: 0, train loss: 4.5830, validation loss: 4.5724
steps: 200, train loss: 3.1530, validation loss: 3.2658
steps: 400, train loss: 3.1468, validation loss: 3.2569
steps: 600, train loss: 3.1501, validation loss: 3.2659
steps: 800, train loss: 3.1509, validation loss: 3.2636
3.148810863494873


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

RuntimeError: The size of tensor a (64) must match the size of tensor b (65) at non-singleton dimension 2